## RetinaNet

Keras-RetinaNet 모델 훈련 및 예측 과정입니다. [Keras-RetinaNet](https://github.com/fizyr/keras-retinanet) 패키지가 필요합니다.

 - 패키지를 다운로드 및 설치합니다.
 ```
 git clone https://github.com/fizyr/keras-retinanet.git
 cd keras-retinanet
 python -m pip install .
 ```

### Train

Keras-RetinaNet 패키지는 커맨드라인 상에서 훈련을 할 수 있는 명령어를 제공합니다. `retinanet-train` 명령어를 사용하시면 됩니다.

 - 기본 사용법: `retinanet-train (데이터셋 종류) (데이터셋 경로)`
     - 데이터셋 종류는 `coco`, `pascal`, `csv`, `oid`, `kitti` 5종류가 있습니다.
 - csv 데이터셋의 경우: `retinanet-train csv (어노테이션 파일 경로) (클래스 파일 경로)`
     - csv 파일은 간단하게 데이터셋을 구성할 수 있는 방법이며 커스텀 데이터셋에 대한 훈련에 적합합니다.
     - 어노테이션 파일은 `이미지/경로.jpg,x1,y1,x2,y2,클래스이름` 양식으로 각 경계 상자에 대해서 한 줄에 하나씩 적으면 됩니다.
     - 이미지 경로를 제외한 정보를 생략할 경우 (쉼표는 빠짐없이 모두 적어줘야 합니다) 음성 샘플로 간주되어 훈련에 사용됩니다.
     - 클래스 파일은 `클래스이름,아이디` 양식으로 클래스 하나당 한 줄에 하나씩 적으면 됩니다.
     - 
 - 추가 옵션:
     - `--weights (경로)` 이미 훈련된 모델 가중치 파일을 불러올 수 있습니다.
     - `--epochs (숫자)` 총 몇 세대동안 훈련할 것인지를 정의합니다. 기본값은 50입니다.
     - `--steps (숫자)` 한 세대동안 몇 장의 이미지를 훈련할 것인지를 정의합니다. 기본값은 10000입니다.
     - `--no-snapshots` 기본값으로 각 세대가 완료될 때마다 훈련된 가중치를 저장합니다. 이 옵션으로 비활성화 할 수 있습니다.
     - 이외에도 여러 옵션이 있습니다. 

자세한 사용방법은 keras-retinanet 패키지 저장소의 설명을 참고하시기 바랍니다.

### Predict

훈련 시와 동일한 환경을 구축해야 합니다. 

In [ ]:
from __future__ import division

import cv2
import numpy as np
import os
from six.moves import zip
import tensorflow as tf
import time

from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.image import  preprocess_image, read_image_bgr, resize_image

#### 데이터셋 설정

아래는 데이터셋 및 객체 검출 설정에 필요한 환경 변수입니다. 사용하는 데이터셋 및 모델에 맞춰서 변경하시기 바랍니다.

 - `classes` : 데이터내에 구성하는 모든 클래스의 정보를 담은 dict입니다. 키는 클래스의 일련변호, 값은 각 클래스의 이름으로 지정해야 합니다.
 - `weight_file_path` : 입력 weight 파일의 경로 및 이름을 지정합니다.
 - `predict_source` : 객체 검출을 할 대상 이미지 파일이 있는 디렉토리를 지정합니다.
 - `predict_result` : 객체 검출 결과가 출력될 디렉토리를 지정합니다. 자동으로 해당 디렉토리를 생성하지 않으므로 미리 생성해두셔야 합니다.
 - `bbox_threshold` : 검출된 경계 상자를 양성(positive)로 판정할 임계값입니다.
 - `resize` : 이미지가 지나치게 크면 리사이즈하여 검출할지를 지정하는 부울 값입니다.

In [ ]:
classes = None
weight_file_path = './retinanet.h5'
predict_source = './images'
predict_result = './results'
bbox_threshold = 0.5

if classes is None:
    classes = {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle',
               5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow',
               10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person',
               15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}

#### 검출 시작

모델을 로드하고 소스 이미지의 객체 검출을 실행하며 그 결과를 출력합니다.

In [ ]:
def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

set_session(get_session())

model = load_model(model_path, custom_objects=custom_objects)

for dir_path, _, filenames in os.walk(predict_source):
    for filename in filenames:
        if not filename.endswith(('.jpg', '.png')):
            continue

        # 이미지 로드
        start = time.time()
        file_path = os.path.join(dir_path, filename)
        print(file_path)
        image = read_image_bgr(file_path)

        # 정규화 및 리사이즈
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # 검출
        _, _, detections = model.predict_on_batch(np.expand_dims(image, axis=0))

        # 결과 후처리
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]
        detections[0, :, :4] /= scale

        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < bbox_threshold:
                continue
            b = detections[0, idx, :4] / scale
            b = b.astype(int)

            print('Label: {}, Score: {}, LTRB of the boundary box: {}, {}, {}, {}'.format(classes[label], score, b[0], b[1], b[2], b[3]))

            cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 3)
            caption = "{} {:.3f}".format(classes[label], score)
            cv2.putText(img, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
            cv2.putText(img, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

        cv2.imwrite(os.path.join(predict_result, filename), img)

        print("Processing time: {}".format(time.time() - start))